In [1]:
import numpy as np
import pandas as pd

C:\Users\Sonat\AppData\Local\Temp\ipykernel_16380\1662815981.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
def power_iteration(A, max_iter=100, tol=1e-6):
    """
    Power Iteration method to find the dominant singular vector of matrix A.

    Parameters:
        A (numpy.ndarray): Input matrix.
        max_iter (int): Maximum number of iterations (default: 100).
        tol (float): Tolerance for convergence (default: 1e-6).

    Returns:
        eig_val (float): The greatest (in absolute value) eigenvalue of A.
        v (numpy.ndarray): nonzero vector which is a corresponding eigenvector of eig_val.
    """
    m, n = A.shape
    v = np.random.rand(n, 1)

    for _ in range(max_iter):
        Av = np.dot(A, v)
        norm_av = np.linalg.norm(Av)
        v_new = Av / norm_av

        # Check for convergence
        if np.linalg.norm(v_new - v) < tol:
            break

        v = v_new

    # Compute the corresponding singular value
    eig_val = np.dot(np.dot(A, v), v) / np.dot(v, v)

    return eig_val, v